# QR Codes Are Magic
While really ugly, QR codes are robust and widely implemented. Like JSON.     

    pip install qrcode
    

    conda install -c conda-forge qrcode

In [2]:
    import qrcode as qrcode, qrcode.image.svg, IPython

    
and you can make QR codes out of arbitrary data which can be read by most cell phones.

Here's a magic which gives you all kinds of toys to play with.

In [3]:
    @IPython.core.magic.register_line_cell_magic
    def qr(line, cell=None):
        line = (line + " " + (cell or "")).strip()
        args = parser.parse_args(line.split())
        q = qrcode.QRCode(version=args.version, error_correction=args.error_correction,
                          box_size=args.size, border=args.border, mask_pattern=args.mask,)
        q.add_data(args.data)
        q.make(fit=True)
        img = None
        if args.output == "png":
            img = q.make_image(fill_color=args.fg, back_color=args.bg)
        elif "svg" in args.output:
            with __import__('io').BytesIO() as out:
                q.make_image(fill_color=args.fg, back_color=args.bg, image_factory={
                    "svg": qrcode.image.svg.SvgImage,
                    "svg-fragment": qrcode.image.svg.SvgFragmentImage,
                    "svg-path": qrcode.image.svg.SvgPathImage,
                }[args.output])._write(out)
                img = IPython.display.SVG(out.getvalue())
        elif args.output == "ascii":
            with IPython.utils.capture.capture_output() as cap: q.print_ascii()

            img = IPython.display.HTML(f"""<pre style="line-height: 1;">\n{cap.stdout}\n</pre>""")
        IPython.display.display(img)

`qrcode` hides it's parser inside a closure, so let's make another one.

In [4]:
    parser = __import__('argparse').ArgumentParser()
    parser.add_argument('data')
    parser.add_argument('-s', '--size', default=10, type=int)
    parser.add_argument('-b', '--border', default=1, type=int)
    parser.add_argument('-e', '--error-correction', default=1, type=int)
    parser.add_argument('-v', '--version', default=1)
    parser.add_argument('--fg', default='black')
    parser.add_argument('--bg', default='white')
    parser.add_argument('-m', '--mask', default=None, type=int)
    parser.add_argument('-o', '--output', default='png', choices=['svg', 'svg-path', 'svg-fragment', 'png', 'ascii']);

## Outputs

### SVG output

In [5]:
    if __name__ == "__main__":
        %qr -o svg-path https://jupyter.org

### Plain Text

A plain text QR to post on your In addition to images, you can generate plain text, which would look great on your BBS!

In [31]:
    if __name__ == "__main__":
        %qr -o ascii https://pydata.org

## Edit your own QR code with an IPython widget.

In [32]:
    if __name__ == "__main__":
        from ipywidgets import interact, ColorPicker
        @interact
        def deathbeds(url="https://deathbeds.github.io", size=(1,10), mask=(0,7), error=(0,3), version=(1,8), border=(0, 10), fore=ColorPicker(value="#0f0"), back=ColorPicker(value="black")):
            %qr -s{size} -m{mask} -e{error} -v{version} -b{border} --fg {fore} --bg {back} {url}

interactive(children=(Text(value='https://deathbeds.github.io', description='url'), IntSlider(value=5, descrip…